[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/beneficial-ownership-sec-python.ipynb)

# Beneficial Ownership from SEC Schedule 13D/G with Python -- Free, No API Key

Use **edgartools** to track beneficial ownership from SEC Schedule 13D and 13G filings -- completely free, no API key or paid subscription required. When an investor acquires more than 5% of a public company, they must file a Schedule 13D (activist) or 13G (passive) disclosing their ownership stake.

**What you'll learn:**
- Find Schedule 13D and 13G filings for any company
- Extract ownership stakes, voting power, and reporting persons
- Read the "purpose of transaction" to understand activist intent
- Distinguish activist investors (13D) from passive holders (13G)
- Track who owns 5%+ of any public company

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## See Who Owns 5%+ of Apple

When an investor acquires more than 5% of a public company, they must file a Schedule 13D or 13G. Parse one to see the ownership details:

In [ ]:
filing = Company("AAPL").get_filings(form="SCHEDULE 13G")[0]
sc13g = filing.obj()

print(f"Issuer:   {sc13g.issuer_info.name}")
print(f"Security: {sc13g.security_info.title}")
print(f"Shares:   {sc13g.total_shares:,}")
print(f"Percent:  {sc13g.total_percent}%")

for person in sc13g.reporting_persons:
    print(f"\n{person.name}:")
    print(f"  Shares owned:      {person.aggregate_amount:,}")
    print(f"  Percent of class:  {person.percent_of_class}%")
    print(f"  Sole voting power: {person.sole_voting_power:,}")

## Schedule 13D: Activist Investors

A Schedule 13D is filed by investors who intend to influence or control the company. Search recent 13D filings to find activist positions:

In [ ]:
filings = get_filings(form="SCHEDULE 13D")
print(f"Recent Schedule 13D filings: {len(filings):,}\n")

# Parse a 13D to see full ownership details
sc13d = filings[0].obj()

print(f"Issuer:       {sc13d.issuer_info.name}")
print(f"Date of event:{sc13d.date_of_event}")
print(f"Total shares: {sc13d.total_shares:,}")
print(f"Percent:      {sc13d.total_percent}%")
print(f"Amendment:    {sc13d.is_amendment}")

for person in sc13d.reporting_persons:
    print(f"\n{person.name} ({person.type_of_reporting_person}):")
    print(f"  Shares:          {person.aggregate_amount:,}")
    print(f"  Percent:         {person.percent_of_class}%")
    print(f"  Sole voting:     {person.sole_voting_power:,}")
    print(f"  Shared voting:   {person.shared_voting_power:,}")

## Purpose of Transaction

The key section of a Schedule 13D is Item 4 -- the "Purpose of Transaction." This reveals why the investor is building a stake:

In [ ]:
purpose = sc13d.items.item4_purpose_of_transaction

if purpose:
    print(f"Purpose of Transaction ({len(purpose):,} chars):\n")
    print(purpose[:1000])
else:
    print("No purpose text disclosed")

## Schedule 13G: Passive Institutional Investors

A Schedule 13G is filed by passive investors (like index funds) who own 5%+ but don't intend to influence the company. These reveal institutional ownership:

In [ ]:
filings_13g = get_filings(form="SCHEDULE 13G")
print(f"Recent Schedule 13G filings: {len(filings_13g):,}\n")

# Find a well-known institutional filer
for f in filings_13g.head(20):
    obj = f.obj()
    if obj and obj.total_shares and obj.total_shares > 1_000_000:
        print(f"Issuer:  {obj.issuer_info.name}")
        print(f"Passive: {obj.is_passive_investor}")
        print(f"Shares:  {obj.total_shares:,}")
        print(f"Percent: {obj.total_percent}%")
        for p in obj.reporting_persons:
            print(f"\n  {p.name}:")
            print(f"    Shares: {p.aggregate_amount:,} ({p.percent_of_class}%)")
            print(f"    Sole dispositive:   {p.sole_dispositive_power:,}")
            print(f"    Shared dispositive: {p.shared_dispositive_power:,}")
        break

## Scan Recent 13D Filings

Build a summary table of recent activist ownership positions:

In [ ]:
filings = get_filings(form="SCHEDULE 13D")
rows = []

for f in filings.head(20):
    obj = f.obj()
    if obj and obj.total_shares and obj.reporting_persons:
        filer = obj.reporting_persons[0].name
        rows.append({
            "Filer": filer[:35],
            "Issuer": obj.issuer_info.name[:30],
            "Shares": f"{obj.total_shares:,}",
            "Percent": f"{obj.total_percent}%",
        })

pd.DataFrame(rows)

## Why EdgarTools?

EdgarTools is free and open-source. Compare tracking beneficial ownership:

**With edgartools (free, no API key):**
```python
sc13d = get_filings(form="SCHEDULE 13D")[0].obj()
sc13d.issuer_info.name          # Target company
sc13d.total_percent             # Ownership stake
sc13d.items.item4_purpose_of_transaction  # Activist intent
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi
api = QueryApi(api_key="YOUR_PAID_API_KEY")
query = {"query": {"query_string": {"query": 'formType:"SC 13D"'}}}
results = api.get_filings(query)  # Raw JSON, still need to parse XML manually
```

With edgartools, Schedule 13D/G data is parsed into clean Python objects -- ownership stakes, voting power, and activist intent all accessible instantly.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Find beneficial ownership filings ──
get_filings(form="SCHEDULE 13D")                # Activist investors
get_filings(form="SCHEDULE 13G")                # Passive investors
get_filings(form=["SCHEDULE 13D", "SCHEDULE 13G"])  # Both
Company("AAPL").get_filings(form="SCHEDULE 13G")    # For a company

# ── Parse a filing ──
sc = filing.obj()                                # Schedule13D or Schedule13G

# ── Issuer (target company) ──
sc.issuer_info.name                              # Company name
sc.issuer_info.cusip                             # CUSIP
sc.security_info.title                           # Security description

# ── Ownership ──
sc.total_shares                                  # Total shares owned
sc.total_percent                                 # Percent of class

# ── Reporting persons ──
for person in sc.reporting_persons:
    person.name                                  # Investor name
    person.aggregate_amount                      # Shares owned
    person.percent_of_class                      # Ownership %
    person.sole_voting_power                     # Sole voting
    person.shared_voting_power                   # Shared voting
    person.sole_dispositive_power                # Sole dispositive
    person.shared_dispositive_power              # Shared dispositive

# ── Activist intent (13D only) ──
sc.items.item4_purpose_of_transaction            # Why they invested
```

## What's Next

You've learned how to track beneficial ownership from SEC Schedule 13D/G filings. Here are related tutorials:

- [Track Insider Trading from SEC Form 4](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)
- [Analyze 13F Institutional Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/13f-institutional-holdings-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)
- [Search and Filter SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/search-sec-filings-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*